<img src=../figures/Brown_logo.svg width=50%>

## Data-Driven Design & Analyses of Structures & Materials (3dasm)

## Lecture 20


### Miguel A. Bessa | <a href = "mailto: miguel_bessa@brown.edu">miguel_bessa@brown.edu</a>  | Associate Professor

## Outline of today

Provide an overview of your Final Project for the course.

This final project is inspired by the following research article:

* **Bessa, M. A., Glowacki, P., & Houlder, M. (2019). Bayesian Machine Learning in Metamaterial Design: Fragile Becomes Supercompressible. Advanced Materials, 31(48), 1–6. https://doi.org/10.1002/adma.201904845**

It focused on transforming a fragile polymer material used in 3D printing into a supercompressible metamaterial by designing its geometry.

The data was generated by Finite Element Analyses (using [ABAQUS](https://www.3ds.com/products/simulia/abaqus) together with `f3dasm`).

However, you will not need to generate the data using ABAQUS. Instead, you will be provided with a dataset. This way, you can focus on the Machine Learning and Optimization tasks and avoid running time-consuming simulations.

## Designing supercompressible metamaterials

<img src=../figures/supercompressible_process.png width=65%, align='left'>

<img src=../figures/supercompressible_schematic.png width=30%, align='right'>

<!--- <br><br><br><br><br><br><br><br><br><br><br><br><br><br><br> --->

* Parametrizing your structure from a concept
* Sampling designs
* Run finite element simulation (ABAQUS)
* Learn the objective for the full range of designs
* Manufacture your found optimum

## Datasets

The supercompressible meta-material is parameterized by **5 geometric parameters and 2 material parameters**:
* The geometry is defined by the top and bottom diameters, $D_1$ and $D_2$, 
* the height $P$
* the cross-section parameters of the vertical longerons: area $A$, 
* moments of inertial $I_x$ and $I_y$, 
* torsional constant $J$,. 
* The isotropic material is defined by its elastic constants: Young's modulus $E$ and shear modulus $G$.

$$
\frac{D_1-D_2}{D_1},\ \frac{P}{D_1},\ \frac{I_x}{D_1^4},\ \frac{I_y}{D_1^4},\ \frac{J}{D_1^4},\ \frac{A}{D_1^2}, \frac{G}{E}
$$

| expression |	parameter name |
| ----------- | --------------- |
| $\frac{D_1-D_2}{D_1}$ |	`ratio_top_diameter`
|$\frac{P}{D_1}$|	`ratio_pitch`
|$\frac{I_x}{D_1^4}$|	`ratio_Ixx`
|$\frac{I_y}{D_1^4}$|	`ratio_Iyy`
|$\frac{J}{D_1^4}$|	`ratio_J`
|$\frac{A}{D_1^2}$|	`ratio_area`
|$\frac{G}{E}$|	`ratio_shear_modulus`



This is a **7-dimensional problem** and learning the response surface may require a **significant amount of training points**

Therefore, you will also consider a **simpler version** of the problem in 3 dimensions, defined by constraining the longerons' cross-section to be circular with diameter $d$, and choosing a particular material, leading to the following 3 features:

$$
\frac{d}{D_1}, \frac{D_2-D_1}{D_1},\ \frac{P}{D_1}
$$

| expression |	parameter name |
| ----------- | --------------- |
$\frac{D_1-D_2}{D_1}$|	`ratio_top_diameter`
$\frac{P}{D_1}$	|`ratio_pitch`
$\frac{d}{D_1}$	|`ratio_d`

In addition to the 7 input variables (or 3 in the simplified case), each design of the supercompressible metamaterial needs to be **perturbed by a small geometric imperfection**.

- Why?

The metamaterial becomes supercompressible because the vertical longerons **buckle**, i.e. undergo an mechanical instability and dramatically coil.

<center><img src=../figures/supercompressible_process.png width=65%></center>

ABAQUS provides a simple way to induce a geometric instability:

1. **Linear buckling analysis** of a metamaterial design considering perfect geometry:
    - Each simulation outputs the buckling load for each mode ("**loads**" output variable), and the maximum displacement of each mode ("**max_disps**" output variable) that are needed to scale the amplitude of the applied geometric imperfections (see below).
    

2. **Postprocess linear buckling analysis** to determine the buckling modes of that design:
    - Use the buckling modes and apply an amplitude value for each of those modes, and add them to create a perturbed geometry;
    - Here we use only the first buckling mode to seed geometric imperfections for each metamaterial design;
    - We only care about a design if its first buckling mode leads to torsion of the top ring (indication of supercompressibility). In that case, we update the "**coilable**" output variable to "coilable = 1". Otherwise, "coilable = 0" and the subsequent postbuckling response prediction of that design is **not** conducted.

Once the linear buckling analysis finishes, and **if that design is supercompressible** ("coilable = 1"), then we continue as follows:

3. **Postbuckling analysis** (RIKS) of metamaterials with "coilable = 1":
    - Use the first buckling mode from the linear buckling analysis to create an imperfect geometry whose amplitude is sampled from a lognormal distribution with mean of $4^{\circ}$ and a standard deviation of $1.2^{\circ}$. This creates some aleatoric uncertainty in the data, and makes it possible to predict the postbuckling response by the RIKS method.

4. **Postprocess postbuckling analysis**:
    - Collect 4 outputs for that design: reaction moments "**RM**", reaction forces "**RF**" at the bottom ring; displacements "**U**" and rotations "**UR**" at the top ring (where load is applied).
    - Check maximum strain on longerons, if it is beyond plastic strain update to "coilable = 2" because such design is supercompressible but leads to plastic yielding.
    
Done: this completes one simulation of a particular metamaterial design.

**NOTE**: In the project you will not have to use ABAQUS because you will be provided with the dataset already. I am giving you this information just so you understand how the data was generated. This course is not a Finite Element course 😉

## How do we normally create this dataset?

<img src="../figures/blocks_supercompressible_original.png" width="50%">

Some problems may arise:
* How do we **parametrize** the design efficiently?
* How do we run this in **parallel** or on a **high performance computing cluster**?
* How do we **avoid reinventing the wheel** (e.g. calling Abaqus with python scripts)

## You guessed it! With f3dasm!

<img src="../figures/supercompressible_colored.png" width="100%">

- For the 7D case, a dataset with $50000$ experiments that is parametrized by 7 parameters (`supercompressible_7d`) will be provided.
- For the 3D case, a dataset with $1000$ experiments (`supercompressible_3d`) will be provided.


Let's recreate the 3D dataset. First create the  parametrized domain:

In [1]:
from f3dasm.design import Domain

We add the parameters and their respective bounds:

In [2]:
domain = Domain()

domain.add_float('ratio_top_diameter', low=0.0, high=0.8)
domain.add_float('ratio_pitch', low=0.25, high=1.5)
domain.add_float('ratio_d', low=0.004, high=0.073)
domain.add_float(name='imperfection', low=0.0, high=1.0)

The Design of Experiments module is very simple:
- [Sobol sequence sampling](https://salib.readthedocs.io/en/latest/_modules/SALib/sample/sobol_sequence.html) is used for the remaining parameters `'ratio_top_diameter'`, `'ratio_pitch'` and `'ratio_d'`

Note: The amplitude of geometric imperfections is sampled from the previously mentioned lognormal distribution for each design. This acts as a hidden rv, not an input variable!

Still, in  order for us to sample both types of variables, we can split the domain in two subdomains and sample from them individually. This is done with the `Domain.select` method:

In [3]:
domain_imperfection = domain.select('imperfection')
domain_rest = domain.select(['ratio_top_diameter', 'ratio_pitch', 'ratio_d'])

We create two separate `f3dasm.ExperimentData` objects and later join them together!

In [4]:
from f3dasm import ExperimentData

experimentdata_imperfection = ExperimentData(domain=domain_imperfection)
experimentdata_rest = ExperimentData(domain=domain_rest)

An implementation of sobol sequence sampling is available as built-in default, so we can use that sampler right away:

In [5]:
N = 2 # Number of samples of the actual dataset is much larger!!

experimentdata_rest.sample(sampler='sobol', n_samples=N, seed=123)
experimentdata_rest

jobs              input                    
        ratio_top_diameter ratio_pitch ratio_d
0  open                0.0        0.25   0.004
1  open                0.4       0.875  0.0385

For the lognormal distribution, we have to provide our own sampling function `log_normal_sampler`:
- The function requires at least the `Domain` object
- Other parameters can be added like the number of samples (`'n_samples'`), the mean and standard deviation of the lognormal distribution and a seed for reproducibility

In [6]:
import numpy as np

def log_normal_sampler(domain: Domain, n_samples: int, mean: float, sigma: float, seed: int):
    rng = np.random.default_rng(seed) # Create a numpy generator with a random seed
    samples = rng.lognormal(mean=mean, sigma=sigma, size=n_samples) # Create samples from a lognormal distribution
    return samples # return the samples

Now we can sample using this custom function:

In [7]:
experimentdata_imperfection.sample(sampler=log_normal_sampler, n_samples=N, 
                          mean=-2.705021452041446, sigma=0.293560379208524,
                          seed=123)
experimentdata_imperfection

,jobs,input
,,imperfection
0,open,0.050017
1,open,0.060025


Lastly, we join both `f3dasm.ExperimentData` objects together using the `join()` method:

In [8]:
experimentdata = experimentdata_rest.join(experimentdata_imperfection)
experimentdata

jobs              input                                 
        ratio_top_diameter ratio_pitch ratio_d imperfection
0  open                0.0        0.25   0.004     0.050017
1  open                0.4       0.875  0.0385     0.060025

Now we are ready to evaluate our samples with a datagenerator:
The simulation consists of two ABAQUS simulations (one after the other):

* The first one is a linear buckling simulation
* The second one is a RIKS simulation

We split up the components of the simulation in three parts:

* **pre-processing**: combine the design and the other parameters/constants to an input file
* **processing**: run the simulation
* **post-processing**: convert the output of the simulator back to `f3dasm`

<img src="../figures/blocks_supercompressible_datagenerator.png" width="100%">

## Data generation: getting the output data

* As mentioned, we want to use **non-linear finite element analyses** to predict the **complete buckling and post-buckling behavior**. 
* From the analyses, we can understand if a **material is coilable**, compute the **critical buckling stress** $\sigma_{crit}$ (critical buckling load of the linear buckling analysis divided by the area of the bottom ring) and the elastic **energy absorbed** $E_{abs}$ (integral of the effective stress-strain response from the postbuckling simulation).
* Due to unsuccessful simulations, there are **missing points in the datasets** in the data. This is very common is real datasets (simulations sometimes can have convergence issues, etc.).

Pip install `abaqus2py`:

In [19]:
%pip install abaqus2py

Note: you may need to restart the kernel to use updated packages.


Import the `F3DASMABaqusSimulator` from the `abaqus2py` package. This facilitates the integration of f3dasm with Abaqus.

In [9]:
from abaqus2py import F3DASMAbaqusSimulator

**Note**: The notes below show how to get the Python scripts used in Abaqus to create the simulations (4 scripts: buckling, RIKS and respective postprocessing of each of these). Since this is not a Finite Element course, we will not discuss these.

In [10]:
# The following code downloads the scripts for the linear buckling analysis and RIKS analysis to the current working directory

import requests # Used for dowloading files
from pathlib import Path # Used for I/O operations

# Github links to the Python files
lin_buckle_script = 'https://raw.githubusercontent.com/bessagroup/f3dasm/refs/heads/main/studies/fragile_becomes_supercompressible/scripts/supercompressible_lin_buckle.py'
lin_buckle_post_script = 'https://raw.githubusercontent.com/bessagroup/f3dasm/refs/heads/main/studies/fragile_becomes_supercompressible/scripts/supercompressible_lin_buckle_pp.py'
riks_script = 'https://raw.githubusercontent.com/bessagroup/f3dasm/refs/heads/main/studies/fragile_becomes_supercompressible/scripts/supercompressible_riks.py'
riks_post_script = 'https://raw.githubusercontent.com/bessagroup/f3dasm/refs/heads/main/studies/fragile_becomes_supercompressible/scripts/supercompressible_riks_pp.py'

# Define a folder where you want to store the scripts
folder_destination = (Path.cwd() / 'scripts')

# If the folder doesn't exist, create it
folder_destination.mkdir(exist_ok=True)

def download_script(url: str, destination: Path):   
    response = requests.get(url)

    with open(destination, 'wb+') as f:
        f.write(response.content)

        
download_script(lin_buckle_script, folder_destination / 'supercompressible_lin_buckle.py')
download_script(lin_buckle_post_script , folder_destination / 'supercompressible_lin_buckle_pp.py')

download_script(riks_script, folder_destination / 'supercompressible_riks.py')
download_script(riks_post_script, folder_destination / 'supercompressible_riks_pp.py')

Create the linear buckling simulator:

In [11]:
simulator_lin_buckle = F3DASMAbaqusSimulator(py_file='./scripts/supercompressible_lin_buckle.py',
                                 post_py_file='./scripts/supercompressible_lin_buckle_pp.py',
                                 working_directory=Path.cwd() / 'lin_buckle')

The linear buckling simulation requires some additional parameters that are **not** part of the Design of Experiments because they do not matter (we used non-dimensional inputs). Still, they need to be added as `kwargs` so that the simulations can run:

* young_modulus = 3500.0
* n_longerson = 3
* bottom_diameter = 100.0
* ratio_shear_modulus = 0.3677

**Note**: If curious, look at the "supercompressible_lin_buckle.py" script to understand how the simulation is created.

Anyway, recall the `experimentdata` object after we did the Design of Experiments (we used 2 samples):

In [13]:
experimentdata

jobs              input                                 
        ratio_top_diameter ratio_pitch ratio_d imperfection
0  open                0.0        0.25   0.004     0.050017
1  open                0.4       0.875  0.0385     0.060025

And now we simply evaluate the `f3dasm.ExperimentData` by running the ABAQUS linear buckling simulation:

In [14]:
experimentdata.evaluate(data_generator=simulator_lin_buckle, kwargs={'young_modulus': 3500.0, 'n_longerons': 3,
                       'bottom_diameter': 100.0, 'ratio_shear_modulus' :0.3677, 'circular': True})

Abaqus License Manager checked out the following license:
"cae" from Flexnet server abaqus1.lic.tudelft.nl
<16 out of 30 licenses remain available>.
Abaqus License Manager checked out the following license:
"cae" from Flexnet server abaqus1.lic.tudelft.nl
<16 out of 30 licenses remain available>.
/home/martin/Documents/GitHub/f3dasm/src/f3dasm/_src/experimentdata/_data.py:481: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'loads/0.npy' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  self.data.at[index, _column_index] = value
/home/martin/Documents/GitHub/f3dasm/src/f3dasm/_src/experimentdata/_data.py:481: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'max_disps/0.npy' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  self.data.at[index, _colu

Once this finishes, we can see the `experimentdata` object again to verify that the outputs were obtained:

In [15]:
experimentdata

jobs              input                                    output  \
            ratio_top_diameter ratio_pitch ratio_d imperfection coilable   
0  finished                0.0        0.25   0.004     0.050017      1.0   
1  finished                0.4       0.875  0.0385     0.060025      1.0   

                                 \
         loads        max_disps   
0  loads/0.npy  max_disps/0.npy   
1  loads/1.npy  max_disps/1.npy   

                                                      
                                      lin_buckle_odb  
0  /home/martin/Documents/GitHub/3dasm_course/Lec...  
1  /home/martin/Documents/GitHub/3dasm_course/Lec...

Now we need to run the second part of the experiment, i.e. the posbuckling simulation (RIKS simulator):

In [16]:
simulator_riks = F3DASMAbaqusSimulator(py_file='./scripts/supercompressible_riks.py',
                                     post_py_file='./scripts/supercompressible_riks_pp.py',
                                      working_directory=Path.cwd() / 'riks')

Evaluate the `f3dasm.ExperimentData` on the RIKS analysis simulator:

In [17]:
experimentdata.mark_all('open')


experimentdata.evaluate(data_generator=simulator_riks, kwargs={'young_modulus': 3500.0, 'n_longerons': 3,
                       'bottom_diameter': 100.0, 'ratio_shear_modulus' :0.3677, 'circular': True})

Abaqus License Manager checked out the following license:
"cae" from Flexnet server abaqus1.lic.tudelft.nl
<16 out of 30 licenses remain available>.
Abaqus License Manager checked out the following license:
"cae" from Flexnet server abaqus1.lic.tudelft.nl
<16 out of 30 licenses remain available>.
/home/martin/Documents/GitHub/f3dasm/src/f3dasm/_src/experimentdata/_data.py:481: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'RM/0.npy' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  self.data.at[index, _column_index] = value
/home/martin/Documents/GitHub/f3dasm/src/f3dasm/_src/experimentdata/_data.py:481: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'RF/0.npy' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  self.data.at[index, _column_index] 

Once this finishes, we can see the `experimentdata` object again to verify that the outputs for the RIKS simulations were obtained:

In [18]:
experimentdata

jobs              input                                    output  \
            ratio_top_diameter ratio_pitch ratio_d imperfection coilable   
0  finished                0.0        0.25   0.004     0.050017      1.0   
1     error                0.4       0.875  0.0385     0.060025    ERROR   

                                 \
         loads        max_disps   
0  loads/0.npy  max_disps/0.npy   
1        ERROR            ERROR   

                                                                          \
                                      lin_buckle_odb        RM        RF   
0  /home/martin/Documents/GitHub/3dasm_course/Lec...  RM/0.npy  RF/0.npy   
1                                              ERROR     ERROR     ERROR   

                      
         U        UR  
0  U/0.npy  UR/0.npy  
1    ERROR     ERROR

Some simulations will lead to errors, and some will simply not run because the design was determined as not supercompressible ("coilable = 0") and we do not care for those postbuckling responses.

As you can see, this `DataGenerator` is more complicated than a simple "analytical function" like the Car Stopping Distance problem, or the benchmark functions that are included in `f3dasm`. Yet, the data-driven process follows the same logic, independently of how advanced each of its modules is.

## What about your Final project?

Your Final Project starts now! Now that you know how the two datasets for the supercompressible metamaterial were created, you can put your Machine Learning and Optimization knowledge into practice...

* The **coilable** output can be used to train **classification models**
* The **critical buckling load** and elastic **energy absorbed** can be used to train **regression models**
* **Optimization methods** can find the best designs (after you did classification and regression).

<img src=../figures/supercompressible_regression_classification.png width=50%>

You have all the tools you need to start the project.

The remaining lectures in the course will provide you more knowledge by covering:

* Classification machine learning models

* Optimization methods (introduction)

* Artificial neural network models

But you are strongly encouraged to train and test **new machine learning** models and **optimization** methods!

You do not need to be confined to the methods we cover in the course because you have the basic theory to do whatever you want!

### See you next class

Have fun!